In [1]:
# declaraciones de librerías
using DataStructures

In [12]:
type gPaths
    nVertices::Int64
    arcos::Array{Int64,2} #arco[i,j] indica la distancia entre i y j 
end

type arcoPaths
    origen::Int64
    destino::Int64
    longitud::Int64
end

type gPathsSparse
    nVertices::Int64
    nArcos::Int64
    pOrigen::Array{Int64,1}
    arcos::Array{arcoPaths,1} #arco[i,j] indica la distancia entre i y j 
end

function generarGrafoDense(n::Int64,maxValue::Int64)
    a=Array{Int64}(n,n)
    rand!(a,1:maxValue)
    for i in 1:n
        a[i,i]=0
    end
    return gPaths(n,a) 
end

function generarGrafoSparse(n::Int64,sp::Float64,maxValue::Int64)
    a=arcoPaths[]
    pOrigen=Int64[]
    nArcos=1
    for i in 1:n
        push!(pOrigen,nArcos)
        for j in 1:n
            if i!=j
                if rand()<sp
                    push!(a,arcoPaths(i,j,rand(1:maxValue)))
                    nArcos += 1
                end
            end
        end
    end
    push!(pOrigen,nArcos)
    return gPathsSparse(n,nArcos-1,pOrigen,a) 
end

function fromDenseToSparse(g::gPaths)
    a=arcoPaths[]
    pOrigen=Int64[]
    nArcos=1
    for i in 1:g.nVertices
        push!(pOrigen,nArcos)
        for j in 1:g.nVertices
            if g.arcos[i,j]!=0
                push!(a,arcoPaths(i,j,g.arcos[i,j]))
                nArcos += 1
            end
        end
    end    
    push!(pOrigen,nArcos)        
    return gPathsSparse(g.nVertices,nArcos-1,pOrigen,a)
end

function fromSparseToDense(g::gPathsSparse)
    ∞=1000000000
    a=Array{Int64}(g.nVertices,g.nVertices)
    fill!(a,∞)
    for i in 1:g.nArcos
        a[g.arcos[i].origen,g.arcos[i].destino]=g.arcos[i].longitud
    end
    return gPaths(g.nVertices,a)
end

fromSparseToDense (generic function with 1 method)

# Djikstra

In [13]:
function dijkstraDense(g::gPaths,origin::Int64)
    ∞=1000000
    π=zeros(Int64,g.nVertices)
    l=Array{Int64}(g.nVertices)
    fill!(l,∞)
    l[origin]=0
    Q=PriorityQueue()
    enqueue!(Q,origin,0)
    while isempty(Q)==false
        v=dequeue!(Q)
        for w in 1:g.nVertices
            if l[w] > (l[v]+g.arcos[v,w])
                l[w]=l[v]+g.arcos[v,w]
                π[w]=v
                if haskey(Q,w)
                    Q[w]=l[w]
                else
                    enqueue!(Q,w,l[w])
                end
            end
        end
    end
    return l
end

function dijkstraSparse(g::gPathsSparse,origin::Int64)
    ∞=1000000
    π=zeros(Int64,g.nVertices)
    l=Array{Int64}(g.nVertices)
    fill!(l,∞)
    l[origin]=0
    Q=PriorityQueue()
    enqueue!(Q,origin,0)
    while isempty(Q)==false
        v=dequeue!(Q)
        for c in g.pOrigen[v]:g.pOrigen[v+1]-1
            w=g.arcos[c].destino
            longitud=g.arcos[c].longitud
            if l[w] > (l[v]+longitud)
                l[w]=l[v]+longitud
                π[w]=v
                if haskey(Q,w)
                    Q[w]=l[w]
                else
                    enqueue!(Q,w,l[w])
                end
            end
        end
    end
    return l
end

dijkstraSparse (generic function with 1 method)

In [14]:
G=generarGrafoDense(2000,1000);
@time(dijkstraDense(G,1));
#l=dijkstraDense(G,1)
#println("in: ",l)
gS=fromDenseToSparse(G);
@time(dijkstraSparse(gS,1));
#lS=dijkstraSparse(gS,1)
#println("in: ",lS)


  0.699061 seconds (7.14 M allocations: 109.532 MB, 3.81% gc time)
  1.837482 seconds (19.14 M allocations: 353.596 MB, 22.02% gc time)


In [ ]:
gS=generarGrafoSparse(2500,0.25,1000)
@time(dijkstraSparse(gS,1))
G=fromSparseToDense(gS)
@time(dijkstraDense(G,1))